In [2]:
import numpy as np
import pandas as pd
from scipy.sparse import load_npz  # Zum Laden der speicheroptimierten Distanzmatrix
from sklearn.cluster import DBSCAN  # DBSCAN-Algorithmus
from sklearn.metrics import davies_bouldin_score  # Davies-Bouldin-Index zur Clusterbewertung
from sklearn.metrics import silhouette_score  # Silhouette-Score zur Clusterbewertung
from scipy.spatial.distance import cdist  # Zur Berechnung der Distanzen zwischen den Clusterzentren
import joblib  # Zum Speichern des Modells

# Die ersten 45.000 Zeilen der Daten laden
df_encoded = pd.read_csv("dbscan_encoded_data.csv").iloc[:45000]

num_samples = len(df_encoded)  # Anzahl der Samples
batch_size = 100  # Batch-Größe für die Verarbeitung
num_batches = (num_samples // batch_size) + 1  # Berechne die Anzahl der Batches

# ------------------------------------
# Verbesserte Epsilon-Bestimmung
# ------------------------------------
k = 10  # Anzahl der nächsten Nachbarn zur Bestimmung von Epsilon
distance_samples = []  # Liste zur Speicherung der Distanzen der k-nächsten Nachbarn

# Ziehe Stichproben aus mehreren Batches
for batch in range(num_batches):
    try:
        # Lade die Distanzmatrix für den aktuellen Batch
        batch_matrix = load_npz(f"combined_distance_batch_{batch}.npz").toarray()
        batch_matrix = batch_matrix[:batch_size, :batch_size]  # Sicherstellen, dass die Matrix quadratisch ist
        
        # Zufällige Stichprobe von Punkten im aktuellen Batch ziehen
        num_sample_points = min(100, batch_matrix.shape[0])  # Maximal 100 Punkte pro Batch
        sample_indices = np.random.choice(batch_matrix.shape[0], num_sample_points, replace=False)
        
        # Hole die Distanzen der k-nächsten Nachbarn
        sample_values = batch_matrix[sample_indices, k - 1]  # k-Nächster Nachbar
        distance_samples.extend(sample_values)  # Füge die Distanzen zur Liste hinzu

    except FileNotFoundError:
        continue

# Bestimme das 95%-Quantil der Distanzen als optimalen Epsilon-Wert
epsilon = np.percentile(distance_samples, 95)
print(f"Optimaler Epsilon-Wert: {epsilon}")

# --- DBSCAN schrittweise auf Batches anwenden ---
cluster_labels = np.full(num_samples, -1, dtype=int)  # Alle Punkte initial als Noise (-1)

cluster_id = 0  # Start-ID für die Cluster
for batch in range(num_batches):
    try:
        # Lade die Distanzmatrix für den aktuellen Batch
        batch_matrix = load_npz(f"combined_distance_batch_{batch}.npz").toarray()
        batch_matrix = batch_matrix[:batch_size, :batch_size]  # Sicherstellen, dass die Matrix quadratisch ist
        
        # Wende DBSCAN auf diesen Batch an
        dbscan = DBSCAN(eps=epsilon, min_samples=30, metric="precomputed")  # Epsilon und min_samples definieren den Cluster
        batch_labels = dbscan.fit_predict(batch_matrix)  # Cluster für den Batch vorhersagen

        # Cluster-Indizes anpassen, sodass sie global fortlaufend sind
        batch_labels[batch_labels != -1] += cluster_id
        cluster_id = max(cluster_id, np.max(batch_labels) + 1)  # Update der globalen Cluster-ID

        # Bestimme den Start- und End-Index für den aktuellen Batch
        start_idx = batch * batch_size
        end_idx = min((batch + 1) * batch_size, num_samples)

        # Cluster-Zuordnungen aktualisieren
        if start_idx < num_samples:
            cluster_labels[start_idx:end_idx] = batch_labels[: end_idx - start_idx]

    except FileNotFoundError:
        continue

# Füge die Clusterzuordnungen zum DataFrame hinzu
df_encoded["Cluster"] = cluster_labels

print("Länge von cluster_labels:", len(cluster_labels))

# Zählen der Cluster (Noise hat den Wert -1)
num_clusters = len(np.unique(cluster_labels[cluster_labels != -1]))
print(f"Anzahl der Cluster: {num_clusters}")

# --- Cluster zusammenführen ---
# Berechne die Mittelwerte für die Merkmale je Cluster (außer Koordinaten)
cluster_means = df_encoded.groupby("Cluster").mean()

# Berechne paarweise Distanzen zwischen den Cluster-Mittelwerten
cluster_distances = cdist(cluster_means, cluster_means, metric='euclidean')

# Definiere eine Schwelle für ähnliche Cluster
merge_threshold = 0.42  # Schwelle für Ähnlichkeit (niedrigere Werte führen zu mehr Clusterzusammenführungen)

# Identifiziere ähnliche Cluster
similar_clusters = np.where(cluster_distances < merge_threshold)

# Initialisiere eine Zuordnung für die Cluster (jeder Cluster ist anfangs sich selbst zugeordnet)
cluster_mapping = {cluster: cluster for cluster in np.unique(cluster_labels)}

# Zusammenführen der ähnlichen Cluster
for i, j in zip(*similar_clusters):
    if i < j:  # Vermeide doppelte Einträge
        cluster_mapping[j] = cluster_mapping[i]  # Cluster j wird mit Cluster i zusammengeführt

# Wende die neue Cluster-Zuordnung auf den DataFrame an
df_encoded["Merged_Cluster"] = df_encoded["Cluster"].replace(cluster_mapping)

# Berechnung des Silhouette-Scores für die zusammengeführten Cluster
# Noise-Punkte (mit -1) werden aus den Berechnungen ausgeschlossen
valid_labels = df_encoded["Merged_Cluster"] != -1
sil_score = silhouette_score(df_encoded[valid_labels], df_encoded.loc[valid_labels, "Merged_Cluster"], metric="euclidean")
print(f"Silhouetten-Koeffizient für die gemergten Cluster: {sil_score}")

# Anzahl der zusammengeführten Cluster ausgeben
num_merged_clusters = df_encoded["Merged_Cluster"].nunique()
print(f"Anzahl der neuen zusammengeführten Cluster: {num_merged_clusters}")

# Speichern der gemergten Cluster-Daten als CSV
df_encoded.to_csv("dbscan_merged_clusters.csv", index=False)
print("Gemergte Cluster-Daten gespeichert!")

# Speichern der gemergten Cluster als .npy-Datei
np.save("dbscan_merged_clusters.npy", df_encoded["Merged_Cluster"].values)
print("Gemergte Cluster-Daten als .npy gespeichert!")

# ------------------------------------
# Davies-Bouldin-Index berechnen auf den gemergten Cluster
# ------------------------------------
# Clusterzentren der gemergten Cluster
merged_cluster_means = df_encoded.groupby("Merged_Cluster").mean()

# Berechne paarweise Distanzen zwischen den Mittelwerten der gemergten Cluster
merged_cluster_distances = cdist(merged_cluster_means, merged_cluster_means, metric='euclidean')

# Berechne den Davies-Bouldin-Index für die gemergten Cluster
dbi_score = davies_bouldin_score(df_encoded[merged_cluster_means.columns], df_encoded["Merged_Cluster"])
print(f'Davies-Bouldin-Index (gemergte Cluster): {dbi_score}')

# ------------------------------------
# --- Modell speichern ---
# ------------------------------------
# Speichern des DBSCAN-Modells
joblib.dump(dbscan, 'dbscan_model.pkl')
print("Das DBSCAN-Modell wurde gespeichert!")


Optimaler Epsilon-Wert: 15.074167823791504
Länge von cluster_labels: 45000
Anzahl der Cluster: 450
Silhouetten-Koeffizient für die gemergten Cluster: 0.3010570165284466
Anzahl der neuen zusammengeführten Cluster: 14
Gemergte Cluster-Daten gespeichert!
Gemergte Cluster-Daten als .npy gespeichert!
Davies-Bouldin-Index (gemergte Cluster): 7.587476521937715
Das DBSCAN-Modell wurde gespeichert!
